In [1]:
!pip install nibabel

In [9]:
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
import nibabel as nib
from models import unet
from utils import min_max_normalization

In [3]:
def dice_coef(y_true, y_pred):
    ''' Metric used for CNN training'''
    smooth = 1.0 #CNN dice coefficient smooth
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    ''' Loss function'''
    return -(dice_coef(y_true, y_pred)) # try negative log of the DICE* (-tf.math.log())

In [4]:
def pad_volume(data,multiple):
    _,d2,d3 = data.shape
    rem2 = multiple - d2%multiple 
    rem3 = multiple - d3%multiple
    return np.pad(data,pad_width = ((0,0),(0,rem2),(0,rem3)))

In [5]:
data_path = "C:\\Users\\David Cooksley\\Documents\\School\\ENEL645\\Project\\enel645-final-project\\unet-segmentation\\Data\\Original"
test_files = "C:\\Users\\David Cooksley\\Documents\\School\\ENEL645\\Project\\enel645-final-project\\unet-segmentation\\Data\\test-set.txt"
test_files = np.loadtxt(test_files,dtype = str)
test_files = [os.path.join(data_path, f + ".gz") for f in test_files]

label01_path = "C:\\Users\\David Cooksley\\Documents\\School\\ENEL645\\Project\\enel645-final-project\\unet-segmentation\\Data\\Brain-masks"
label01_files = "C:\\Users\\David Cooksley\\Documents\\School\\ENEL645\\Project\\enel645-final-project\\unet-segmentation\\Data\\test-set.txt"
label01_files = np.loadtxt(label01_files,dtype = str)
label01_files = [os.path.join(label01_path, f.split('.')[0] + "_staple.nii.gz") for f in label01_files]

label02_path = "C:\\Users\\David Cooksley\\Documents\\School\\ENEL645\\Project\\enel645-final-project\\unet-segmentation\\Data\\WM-GM-CSF"
label02_files = "C:\\Users\\David Cooksley\\Documents\\School\\ENEL645\\Project\\enel645-final-project\\unet-segmentation\\Data\\test-set.txt"
label02_files = np.loadtxt(label02_files,dtype = str)
label02_files = [os.path.join(label02_path, f + ".gz") for f in label02_files]

files = zip(test_files, label01_files, label02_files)

multiple = 2**4 # the network has 4 max-pooling layers

In [6]:
model_name = "unet_multitask.h5"
model = unet(input_shape=(None, None, 1))
model.load_weights(model_name)
model.compile(loss = dice_coef_loss, optimizer=Adam())

In [8]:
image = nib.load(label01_files[0]).get_fdata()
image.shape

ERROR! Session/line number was not unique in database. History logging moved to new session 472


(224, 256, 256)

In [9]:
idx = 120
scores01 = np.zeros(48)
scores02 = np.zeros(48)

for i, (ii, jj, kk) in enumerate(zip(test_files, label01_files, label02_files)):
    data = nib.load(ii).get_fdata()
    label01 = nib.load(jj).get_fdata()
    label02 = nib.load(kk).get_fdata()
    
    data = min_max_normalization(data)
    data = data.transpose(2,0,1) # axial slices first
    label01 = label01.transpose(2,0,1) # axial slices first
    label02 = label02.transpose(2,0,1) # axial slices first
    
    
    H,W,Z = data.shape
    data_padded = pad_volume(data,16)
    pred01,pred02 = model.predict(data_padded[:,:,:,np.newaxis])
    pred01 = pred01[:,:W,:Z,0] > 0.5
    pred01 = pred01.astype(np.double)
    pred02 = pred02[:,:W,:Z,:].argmax(axis = -1)*pred01
    
    # print("Finished loop: " + str(i))
    scores01[i] = dice_coef_loss(label01, pred01)
    scores02[i] = dice_coef_loss(label02, pred02)
    print("Finished loop: " + str(i))
    print("Score 01: " + str(scores01[i]))
    print("Score 02: " + str(scores02[i]))
    
    # print(score01)
    
    # score01 = model.evaluate(data_padded[:,:,:,np.newaxis], 
    # score02 = model.evaluate(data_padded[:,:,:,np.newaxis], 
    # plt.figure()
    # plt.imshow(data[idx], cmap = "gray")
    # plt.imshow(pred01[idx], alpha = 0.5)
    # plt.show()
    # plt.figure()
    # plt.imshow(data[idx], cmap = "gray")
    # plt.imshow(pred02[idx], alpha = 0.5)
    # plt.show()    

Finished loop: 0
Score 01: -0.9836050186739957
Score 02: -0.9836050186739957
Finished loop: 1
Score 01: -0.9865523316648409
Score 02: -0.9865523316648409
Finished loop: 2
Score 01: -0.980557628949921
Score 02: -0.980557628949921
Finished loop: 3
Score 01: -0.985667770052149
Score 02: -0.985667770052149
Finished loop: 4
Score 01: -0.9820876078264036
Score 02: -0.9820876078264036
Finished loop: 5
Score 01: -0.9865580153384426
Score 02: -0.9865580153384426
Finished loop: 6
Score 01: -0.9854390853623822
Score 02: -0.9854390853623822
Finished loop: 7
Score 01: -0.9851206906521883
Score 02: -0.9851206906521883
Finished loop: 8
Score 01: -0.9813380378706549
Score 02: -0.9813380378706549
Finished loop: 9
Score 01: -0.9817798406409574
Score 02: -0.9817798406409574
Finished loop: 10
Score 01: -0.9817270108995981
Score 02: -0.9817270108995981
Finished loop: 11
Score 01: -0.9858257536336522
Score 02: -0.9858257536336522
Finished loop: 12
Score 01: -0.9861979041685011
Score 02: -0.9861979041685011


In [10]:
with open('scores01.npy', 'wb') as f:
    np.save(f, scores01)
with open('scores02.npy', 'wb') as f:
    np.save(f, scores02)

In [50]:
file_names = np.loadtxt("C:\\Users\\David Cooksley\\Documents\\School\\ENEL645\\Project\\enel645-final-project\\unet-segmentation\\Data\\test-set.txt",dtype = str)
file_names = [f.split(".")[0] for f in file_names]

manufacturer = [f.split("_")[1] for f in file_names]
field_strength = [f.split("_")[2] for f in file_names]
sex = [f.split("_")[4] for f in file_names]

In [51]:
scores01 = -np.load("scores01.npy")
scores02 = -np.load("scores02.npy")

In [61]:
d = {'manufacturer':manufacturer, 'field_strength':field_strength, 'sex':sex, 'staple_score':scores01, 'wm_gm_csf_score':scores02 }

In [62]:
score_frame = pd.DataFrame(data=d)
score_frame

,manufacturer,field_strength,sex,staple_score,wm_gm_csf_score
0,siemens,3,M,0.983605,2.423160
1,siemens,3,M,0.986552,2.416392
2,siemens,3,M,0.980558,2.409079
3,siemens,3,M,0.985668,2.423788
4,siemens,3,F,0.982088,2.424768
5,siemens,3,F,0.986558,2.411977
6,siemens,3,F,0.985439,2.413691
7,siemens,3,F,0.985121,2.436038
8,philips,3,M,0.981338,2.389270
9,philips,3,M,0.981780,2.391772


In [63]:
score_frame.groupby('sex').mean()

,staple_score,wm_gm_csf_score
sex,,
F,0.986621,2.430370
M,0.984969,2.429011


ERROR! Session/line number was not unique in database. History logging moved to new session 483


In [64]:
score_frame.groupby('manufacturer').mean()

,staple_score,wm_gm_csf_score
manufacturer,,
ge,0.986340,2.427307
philips,0.986013,2.432121
siemens,0.985032,2.429643


In [65]:
score_frame.groupby('field_strength').mean()

,staple_score,wm_gm_csf_score
field_strength,,
15,0.986372,2.437405
3,0.985218,2.421976
